In [38]:
import numpy as np
import random
import os
import random
import glob

data_a_list = data_b_list = np.hstack((np.arange(2**15), np.arange(-1, -2**15-1, -1)))

n_data = 50000
n_cycle = n_data + 10000
data_a_select = np.random.choice(data_a_list, size=n_data, replace=True)
data_b_select = np.random.choice(data_b_list, size=n_data, replace=True)
shift_select = np.random.choice(data_b_list[:17], size=n_data, replace=True)


def generate_array_with_ones_and_zeros(num_ones, total_length):
    # 確保指定的數量不超過陣列的總長度
    if num_ones > total_length:
        raise ValueError("Number of ones cannot exceed total array length.")
    
    # 生成指定數量的1
    array = [1] * num_ones
    
    # 剩下的空間用0填充
    zeros_to_add = total_length - num_ones
    array += [0] * zeros_to_add
    
    # 將陣列隨機打亂
    random.shuffle(array)
    
    return array

VALID = generate_array_with_ones_and_zeros(n_data, n_cycle)
with open("VALID.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(map(str, VALID)) + "\n")


Signed Addition

In [39]:
inst = '0000'
answer = data_a_select + data_b_select
upper_limit = answer > 2**15-1
lower_limit = answer < -2**15

answer[upper_limit] = 2**15-1
answer[lower_limit] = -2**15
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)


combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("add_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("add_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Signed Subtraction 

In [40]:
inst = '0001'
answer = data_a_select - data_b_select
upper_limit = answer > 2**15-1
lower_limit = answer < -2**15

answer[upper_limit] = 2**15-1
answer[lower_limit] = -2**15
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)


combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("sub_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("sub_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Signed Multiplication

In [41]:
inst = '0010'

answer = np.floor(((data_a_select * data_b_select) / (2**10)) + 0.5)   # rounding, additional 10 bit fraction
upper_limit = answer > 2**15-1
lower_limit = answer < -2**15

answer[upper_limit] = 2**15-1
answer[lower_limit] = -2**15
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)


combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("mul_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("mul_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Signed Accumulation 

In [204]:
inst = '0011'

n_data_acc = 4000   # couldn't be too large because memory would overflow.
addr_select = np.random.choice(data_a_list[:16], size=n_data_acc, replace=True)
data_b_select_acc = np.random.choice(data_b_list, size=n_data_acc, replace=True)

data_acc = np.zeros(16)
answer = []
for addr, data in zip(addr_select, data_b_select_acc):
    data_acc[addr] += data
    answer.append(data_acc[addr])
    # if abs(data_acc[addr]) > 2**19: 
        # print("Error! Exceed limit")
        # break
answer = np.array(answer)
print(max(answer), min(answer), 2**19)

upper_limit = answer > 2**15-1
lower_limit = answer < -2**15
answer[upper_limit] = 2**15-1
answer[lower_limit] = -2**15
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(addr_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select_acc), width=16)


combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("acc_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("acc_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")
    

521682.0 -513829.0 524288
466079.0 -454573.0 524288


Softplus

In [43]:
inst = '0100'

answer = np.zeros(n_data)

range1 = data_a_select >= (2<<10)                                   # x>=2
range2 = ((0) <= data_a_select) & (data_a_select <= (2<<10))        # 0<=x<=2
range3 = ((-1<<10) <= data_a_select) & (data_a_select <= (0))       # -1<=x<=0
range4 = ((-2<<10) <= data_a_select) & (data_a_select <= (-1<<10))  # -2<=x<=-1
range5 = ((-3<<10) <= data_a_select) & (data_a_select <= (-2<<10))  # -3<=x<=-2
range6 = data_a_select <= (-3<<10)                                  # x<=-3

answer[range1] = data_a_select[range1]   # y = x
answer[range2] = (2*data_a_select[range2]+(2<<10))/3   # y = (2*x+2)/3
answer[range3] = (data_a_select[range3]+(2<<10))/3   # y = (x+2)/3
answer[range4] = (2*data_a_select[range4]+(5<<10))/9   # y = (2*x+5)/9
answer[range5] = (data_a_select[range5]+(3<<10))/9   # y = (x+3)/9
answer[range6] = 0   # y = 0
answer = np.floor(answer + 0.5)    # rounding       

upper_limit = answer > 2**15-1
lower_limit = answer < -2**15
answer[upper_limit] = 2**15-1
answer[lower_limit] = -2**15
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)


combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("soft_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("soft_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

XOR

In [44]:
inst = '0101'

answer = data_a_select ^ data_b_select 

answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)

combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("xor_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("xor_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Arithmetic Right Shift 

In [45]:
inst = '0110'

answer = data_a_select >> shift_select 

answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)
data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(shift_select), width=16)

combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("asr_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("asr_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Left Rotation 

In [46]:
inst = '0111'

def left_rotate_str(binary_str, n):
    """
    將二進位字串進行左旋轉
    :param binary_str: 代表二進位的字串
    :param n: 旋轉的位數
    :return: 左旋轉後的字串
    """
    return binary_str[n:] + binary_str[:n]  # 左旋轉通過字串拼接實現

data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(shift_select), width=16)
answer_bin = np.vectorize(left_rotate_str)(data_a_bin, shift_select)

combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("lrot_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("lrot_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Count Leading Zeros

In [47]:
inst = '1000'

def count_leading_zeros(binary_str):
    """
    計算二進位字串中的前導零數量
    :param binary_str: 二進位字串
    :return: 前導零的數量
    """
    return len(binary_str) - len(binary_str.lstrip('0'))

data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)
answer = np.vectorize(count_leading_zeros)(data_a_bin)
answer_bin = np.vectorize(np.binary_repr)(np.int16(answer), width=16)

combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("clz_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("clz_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

Reverse Match4

In [48]:
inst = '1001'

def reverse_match4(i_data_a, i_data_b):
    """
    將兩個16位二進位字串進行反向4位匹配操作
    :param i_data_a: 第一個16位二進位字串
    :param i_data_b: 第二個16位二進位字串
    :return: 匹配結果二進位字串
    """
    o_data = ['0', '0', '0']  # 最前面3位直接設置為0
    
    # 遍歷 i_data_a 的位，並進行4位反向匹配，從第4位開始
    for i in range(13):
        # 取i_data_a的第i到i+3位
        segment_a = i_data_a[i:i+4]
        # 取i_data_b的反向對應4位 (15-i 至 12-i)
        segment_b = i_data_b[15-i-3:15-i+1]
        
        # 如果匹配則為1，否則為0
        if segment_a == segment_b:
            o_data.append('1')
        else:
            o_data.append('0')
    
    return ''.join(o_data)

data_a_bin = np.vectorize(np.binary_repr)(np.int16(data_a_select), width=16)
data_b_bin = np.vectorize(np.binary_repr)(np.int16(data_b_select), width=16)
answer_bin = np.vectorize(reverse_match4)(data_a_bin, data_b_bin)

combined_input = list(map(lambda a, b: inst + str(a) + str(b), data_a_bin, data_b_bin))
with open("mat4_i.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(combined_input) + "\n")
    
with open("mat4_o.dat", "w", encoding="utf-8") as file:
    file.writelines("\n".join(answer_bin) + "\n")

ALL MIX

In [205]:
def shuffle_lines_in_files(extension):
    input_lines = []
    output_lines = []
    acc_input = []
    acc_output = []
    
    # 遍歷當前資料夾內所有指定副檔名的檔案
    for file_name in glob.glob(f'*.{extension}'):
        if file_name[:3] == 'mix':    # ignore
            continue
        elif file_name[-5] == 'i':    # inupt file
            # 打開檔案並讀取所有行
            with open(file_name, 'r') as file:
                lines = file.readlines()
                if file_name[:3] == 'acc':
                    acc_input.extend(lines)  # 將行添加到 acc_input 列表
                else:
                    input_lines.extend(lines)  # 將行添加到 input_lines 列表
        elif file_name[-5] == 'o':    # output file
            # 打開檔案並讀取所有行
            with open(file_name, 'r') as file:
                lines = file.readlines()
                if file_name[:3] == 'acc':
                    acc_output.extend(lines)  # 將行添加到 acc_output 列表
                else:
                    output_lines.extend(lines)  # 將行添加到 output_lines 列表

    # input、output以相同順序打亂
    assert len(input_lines) == len(output_lines), "input, output長度必須相同"
    assert len(acc_input) == len(acc_output), "input, output長度必須相同"
    combined = list(zip(input_lines, output_lines))
    random.shuffle(combined)
    
    # acc先後順序不變，混入combined
    total_positions = list(range(len(combined) + len(acc_input)))
    insert_positions = sorted(random.sample(total_positions, len(acc_input)))
    remaining_positions  = sorted(set(total_positions) - set(insert_positions))

    # 使用 zip(*...) 解壓，將元組列表解壓回兩個打亂的陣列
    shuffled_input_lines = np.empty(len(total_positions), dtype=object)
    shuffled_output_lines = np.empty(len(total_positions), dtype=object)
    shuffled_input_lines[insert_positions], shuffled_output_lines[insert_positions] = acc_input, acc_output
    shuffled_input_lines[remaining_positions], shuffled_output_lines[remaining_positions] = zip(*combined)

    # write shuffled input, output
    with open('mix_i.dat', 'w') as file:
        file.writelines(shuffled_input_lines)
    with open('mix_o.dat', 'w') as file:
        file.writelines(shuffled_output_lines)
    n_inst = len(input_lines) +len(acc_input)
    n_cycle = n_inst + 10000
    VALID = generate_array_with_ones_and_zeros(n_inst, n_cycle)
    with open("VALID.dat", "w", encoding="utf-8") as file:
        file.writelines("\n".join(map(str, VALID)) + "\n")
    print('Number of instruction:', n_inst)
    print('Number of cycle:', n_cycle)
    

shuffle_lines_in_files('dat')

Number of instruction: 454000
Number of cycle: 464000


2's complements check

In [50]:
def twos_complement_to_int(binary_str):
    # 取得二進位字串的長度
    n = len(binary_str)
    
    # 判斷符號位，如果最左側是 '1'，表示是負數
    if binary_str[0] == '1':
        # 將二進位字串轉換為負數的十進位表示
        return int(binary_str, 2) - (1 << n)
    else:
        # 如果符號位是 '0'，直接轉換為正數
        return int(binary_str, 2)

check_a_int = twos_complement_to_int('0000000000000010')
check_b_int = twos_complement_to_int("0010010011000110")
check_golden_int = twos_complement_to_int('1010110101100000')
check_output_int = twos_complement_to_int('0010010011000110')

check_a_int, check_b_int, check_a_int * check_b_int, check_golden_int, check_output_int

(2, 9414, 18828, -21152, 9414)